In [12]:
import wind_calculations as wc
import panel as pn

In [13]:
pn.extension()








def info_icon(text):
    return pn.pane.Markdown(
        f"<span title='{text}' style='cursor:help; color:#1976d2; font-size:18px;'>ℹ️</span>",
        width=20
    )








def calculate_outputs(rotor_diameter, available_area_km2, spacing_factor, efficiency):
    air_density = wc.air_density_lookup[rotor_diameter]
    wind_speed = wc.wind_speed_lookup[rotor_diameter]
    pd = wc.annual_power_density(wind_speed, air_density)
    area = wc.swept_area(rotor_diameter)
    pk = wc.power_kw(pd, rotor_diameter)
    energy_non_derated = wc.annual_energy_output(pk)
    energy_derated = wc.derated_annual_energy_output(pk, efficiency)
    turbines = wc.possible_turbine_installations(available_area_km2, rotor_diameter, spacing_factor)
    # Derived spacing (center-to-center) between turbines in meters
    spacing_m = rotor_diameter * spacing_factor
    # Site totals
    site_power = pk * turbines
    site_energy_non_derated = energy_non_derated * turbines
    site_energy_derated = energy_derated * turbines
    return {
        'air_density': air_density,
        'wind_speed': wind_speed,
        'power_density': pd,
        'swept_area': area,
        'power_kw': pk,
        'energy_non_derated': energy_non_derated,
        'energy_derated': energy_derated,
        'turbines': turbines,
        'turbine_spacing_m': spacing_m,
        'site_power': site_power,
        'site_energy_non_derated': site_energy_non_derated,
        'site_energy_derated': site_energy_derated,
        'efficiency': efficiency
    }








# Panel widgets
rotor_diameter_dropdown = pn.widgets.Select(
    name="Turbine Rotor Diameter (m)",
    options=[100, 150, 200, 250],
    value=100
)
available_area_input = pn.widgets.IntSlider(
    name="Available Area (km²)",
    start=100,
    end=1000,
    step=50,
    value=200
)




# Spacing factor slider
spacing_factor_slider = pn.widgets.FloatSlider(
    name="Turbine Density Factor (Spacing Factor)",
    start=3.0,
    end=9.0,
    step=0.1,
    value=6.0,
    format='0.00'
)




# Efficiency (derating) input as integer percent
efficiency_input = pn.widgets.IntInput(
    name="Efficiency (derating, %)",
    start=20,
    end=30,
    step=1,
    value=20
)




# Unit selectors (apply to both single-turbine and site outputs)
power_unit_select = pn.widgets.Select(
    name="Power unit",
    options=["kW", "MW", "GW"],
    value="kW",
)
energy_unit_select = pn.widgets.Select(
    name="Energy unit",
    options=["MWh/year", "GWh/year", "TWh/year"],
    value="MWh/year",
)








def _scale_power_kw(val_kw, unit):
    factors = {"kW": 1.0, "MW": 1e-3, "GW": 1e-6}
    return val_kw * factors[unit]




def _scale_energy_mwh(val_mwh, unit):
    factors = {"MWh/year": 1.0, "GWh/year": 1e-3, "TWh/year": 1e-6}
    return val_mwh * factors[unit]








def _fmt_number(val, decimals):
    # Keep thousands separators, and variable decimals
    fmt = f",.{decimals}f"
    return format(val, fmt)








@pn.depends(rotor_diameter_dropdown, available_area_input, spacing_factor_slider, efficiency_input, power_unit_select, energy_unit_select)
def results_panel(rotor_diameter, available_area_km2, spacing_factor, efficiency, p_unit, e_unit):
    # Convert integer percent (20-30) to fraction (0.20-0.30) for calculations
    eff_fraction = efficiency / 100.0
    out = calculate_outputs(rotor_diameter, available_area_km2, spacing_factor, eff_fraction)




    # Original simple row: control + column of air density and wind speed
    widget_row = pn.Row(
        rotor_diameter_dropdown,
        pn.Column(
            pn.Row(
                pn.pane.Markdown("<b>Air Density (kg/m³):</b> {:.2f}".format(out['air_density'])),
                info_icon("Density of air at hub height")
            ),
            pn.Row(
                pn.pane.Markdown("<b>Average Wind Speed (m/s):</b> {:.2f}".format(out['wind_speed'])),
                info_icon("Average wind speed at hub height")
            )
        )
)




    units_row = pn.Row(
        power_unit_select,
        energy_unit_select,
        info_icon("Select units for power and annual energy outputs")
)




    # Section divider
    divider = pn.pane.HTML("<hr style='border:1px solid #bbb; margin:20px 0;'>")




    # Compute scaled values for display
    single_power_scaled = _scale_power_kw(out['power_kw'], p_unit)
    single_energy_nd_scaled = _scale_energy_mwh(out['energy_non_derated'], e_unit)
    single_energy_d_scaled = _scale_energy_mwh(out['energy_derated'], e_unit)




    site_power_scaled = _scale_power_kw(out['site_power'], p_unit)
    site_energy_nd_scaled = _scale_energy_mwh(out['site_energy_non_derated'], e_unit)
    site_energy_d_scaled = _scale_energy_mwh(out['site_energy_derated'], e_unit)




    # Decide decimals (0 for base units, 3 for larger aggregates)
    p_decimals = 0 if p_unit == "kW" else 3
    e_decimals = 0 if e_unit.startswith("MWh") else 3




    # Single Turbine Output row (mean/EPF only)
    single_turbine_row = pn.Column(
        pn.pane.Markdown("## Single Turbine Output"),
        widget_row,
        units_row,
        pn.Row(
            efficiency_input,
            info_icon("Overall efficiency/derating factor applied to annual energy"),
            pn.pane.Markdown("<span style='color:#555;'>Please select a value between 20 and 30</span>")
        ),
        pn.Row(
            pn.Column(
                pn.pane.Markdown("### Mean Power & Energy"),
                pn.Row(
                    pn.pane.Markdown(f"**Swept Area:** {out['swept_area']:,.2f} m²"),
                    info_icon("Area swept by the turbine blades")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Mean Power Density (EPF-adjusted):** {out['power_density']:,} W/m²"),
                    info_icon("Mean power per unit rotor area (EPF-adjusted; based on average wind speed)")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Mean Power (EPF-adjusted):** {_fmt_number(single_power_scaled, p_decimals)} {p_unit}"),
                    info_icon("Mean power based on average wind speed and EPF; not the nameplate/rated maximum")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Annual Energy Output (non-derated):** {_fmt_number(single_energy_nd_scaled, e_decimals)} {e_unit}"),
                    info_icon("Total annual energy output without losses")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Derated Annual Energy Output ({out['efficiency']*100:.1f}% efficiency):** {_fmt_number(single_energy_d_scaled, e_decimals)} {e_unit}"),
                    info_icon("Annual energy output accounting for typical losses")
                )
            )
        )
)




    # Site Output row
    site_row = pn.Column(
        divider,
        pn.pane.Markdown("## Site Output"),
        available_area_input,
        pn.Row(spacing_factor_slider, info_icon("Spacing factor F (typical range: 3-10 for offshore wind farms)")),
        pn.Row(
            pn.Column(
                pn.Row(
                    pn.pane.Markdown(f"**Turbine Density Factor (F):** {spacing_factor:.2f}"),
                    info_icon("User-selected turbine density/spacing factor")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Turbine Spacing:** {out['turbine_spacing_m']:,.0f} m"),
                    info_icon("Center-to-center spacing = rotor diameter × F")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Installed Turbines:** {out['turbines']:,}"),
                    info_icon("Number of turbines that can be installed on site")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Total Mean Power (EPF-adjusted):** {_fmt_number(site_power_scaled, p_decimals)} {p_unit}"),
                    info_icon("Total mean power for all turbines on site (EPF-adjusted)")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Total Annual Energy Output (non-derated):** {_fmt_number(site_energy_nd_scaled, e_decimals)} {e_unit}"),
                    info_icon("Total annual energy output for the site without losses")
                ),
                pn.Row(
                    pn.pane.Markdown(f"**Total Derated Annual Energy Output ({out['efficiency']*100:.1f}% efficiency):** {_fmt_number(site_energy_d_scaled, e_decimals)} {e_unit}"),
                    info_icon("Total annual energy output for the site accounting for typical losses")
                )
            )
        )
)




    return pn.Column(
        single_turbine_row,
        site_row
)








# Add a text title to the app
app_panel = pn.Column(
    pn.pane.Markdown("# Offshore wind calculations"),
    results_panel
)

In [14]:
pn.serve(app_panel)

Launching server at http://localhost:56632
